## Question to Answer: 
Can we classify the heart disease risk in each county in USA based on demographic, socio-economic, health measures data? 

## Dataset: 
3 different datasets all grouped by US State and County.  
1) Dataset on heart disease mortality rate 
2) Dataset on socio-economic and health measures data by county 
3) Dataset on population 

## Machine Learning Model: 
We chose Keras Sequential model.  This model allows a random subset of features to be built and and trained to improve accuracy. 

### Building the workflow 

1) Preprocess the data.  Based on the mortality rate in each county, we classified the data into four classes: 
- Class 1 is the counties in the 1st percentile of the mortality rate distribution 
- Class 2 is the counties that XXX 
- Class 3 XXX 
- Class 4 is the counties above the 3rd percentile of the mortality rate distribution 

2) The Datasets will be merged and encoded when necessary 

3) The target will be the classes of mortality 

4) Split, scale, and fit the data 

5) Predict and assess the model 

In [20]:
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sklearn as skl
import tensorflow as tf
import keras
from sklearn.preprocessing import LabelEncoder

In [21]:
from keras.utils import np_utils

In [27]:
df_combined = pd.read_csv("Data/cleaned_data.csv")
df_combined.head()

,State,County,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,...,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income,Rate_100000,Rate_Level,Area_Name,POP_ESTIMATE
0,AL,Autauga,21,4.7,4.7,18,33,7.2,35,69,...,15,87,40,11,12,59338,422.022222,4,Autauga,55869
1,AL,Baldwin,18,4.2,4.3,17,31,8.0,27,74,...,14,84,42,11,5,57588,321.570588,2,Baldwin,223234
2,AL,Barbour,30,5.4,5.2,22,42,5.6,24,53,...,15,83,32,18,11,34382,461.144444,4,Barbour,24686
3,AL,Bibb,19,4.6,4.6,19,38,7.8,34,16,...,10,85,50,15,3,46064,393.036364,3,Bibb,22394
4,AL,Blount,22,4.9,4.9,19,34,8.4,30,16,...,11,86,59,17,3,50412,387.481818,3,Blount,57826


In [29]:
# Drop State and county columns as they do not add weight to the ML model. 
#Drop "Rate level" as the target 
X = df_combined.copy()
X = X.drop(["State", "County", "Area_Name","Rate_Level"], 1)
X.head()

,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,Percent_Excessive_Drinking,Percent_Uninsured,...,Polution_Average_Daily_PM2.5,Presence_of_Water_Violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income,Rate_100000,POP_ESTIMATE
0,21,4.7,4.7,18,33,7.2,35,69,15,9,...,11.7,0,15,87,40,11,12,59338,422.022222,55869
1,18,4.2,4.3,17,31,8.0,27,74,18,11,...,10.3,0,14,84,42,11,5,57588,321.570588,223234
2,30,5.4,5.2,22,42,5.6,24,53,13,12,...,11.5,0,15,83,32,18,11,34382,461.144444,24686
3,19,4.6,4.6,19,38,7.8,34,16,16,10,...,11.2,0,10,85,50,15,3,46064,393.036364,22394
4,22,4.9,4.9,19,34,8.4,30,16,14,13,...,11.7,0,11,86,59,17,3,50412,387.481818,57826


In [31]:
dataset = df_combined.values
y = dataset[:,32]
y

array([4, 2, 4, ..., 2, 2, 3], dtype=object)

In [32]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, random_state=78)

In [34]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Create a keras sequential model 
nn_model = tf.keras.models.Sequential()

In [36]:
input_features = len(X_train_scaled [0])
units_layer1 = input_features*2
nn_model.add(tf.keras.layers.Dense(units=units_layer1, activation="relu", input_dim=input_features))
nn_model.add(tf.keras.layers.Dense(units =4, activation = "relu"))
nn_model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])


In [37]:
#train the model 
fit_model = nn_model.fit(X_train_scaled, y_train, epochs = 50)
#evaluate model accuracy
model_loss, model_accuracy =nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Train on 2209 samples
Epoch 1/50
2209/2209 [==============================] - 1s 348us/sample - loss: nan - accuracy: 0.2254
Epoch 2/50
2209/2209 [==============================] - 0s 67us/sample - loss: nan - accuracy: 0.2268
Epoch 3/50
2209/2209 [==============================] - 0s 93us/sample - loss: nan - accuracy: 0.2268
Epoch 4/50
2209/2209 [==============================] - 0s 85us/sample - loss: nan - accuracy: 0.2268
Epoch 5/50
2209/2209 [==============================] - 0s 67us/sample - loss: nan - accuracy: 0.2268
Epoch 6/50
2209/2209 [==============================] - 0s 73us/sample - loss: nan - accuracy: 0.2268
Epoch 7/50
2209/2209 [==============================] - 0s 67us/sample - loss: nan - accuracy: 0.2268
Epoch 8/50
2209/2209 [==============================] - 0s 68us/sample - loss: nan - accuracy: 0.2268
Epoch 9/50
2209/2209 [==============================] - 0s 67us/sample - loss: nan - accuracy: 0.2268
Epoch 10/50
2209/2209 [==============================] - 0s

In [38]:
#Drop actual mortality rate column to test model accuracy
X_droprate = X.drop("Rate_100000", 1)
X_droprate.head()

,Percent_Fair_or_Poor_Health,Average_Number_of_Physically_Unhealthy_Days,Average_Number_of_Mentally_Unhealthy_Days,Percent_Smokers,Percent_Adults_with_Obesity,Food_Environment_Index,Percent_Physically_Inactive,Percent_With_Access_to_Exercise_Opportunities,Percent_Excessive_Drinking,Percent_Uninsured,...,Violent_Crime_Rate,Polution_Average_Daily_PM2.5,Presence_of_Water_Violation,Percent_Severe_Housing_Problems,Percent_Drive_Alone_to_Work,Percent_Long_Commute,Percent_Adults_with_Diabetes,Percent_Limited_Access_to_Healthy_Foods,Median_Household_Income,POP_ESTIMATE
0,21,4.7,4.7,18,33,7.2,35,69,15,9,...,272,11.7,0,15,87,40,11,12,59338,55869
1,18,4.2,4.3,17,31,8.0,27,74,18,11,...,204,10.3,0,14,84,42,11,5,57588,223234
2,30,5.4,5.2,22,42,5.6,24,53,13,12,...,414,11.5,0,15,83,32,18,11,34382,24686
3,19,4.6,4.6,19,38,7.8,34,16,16,10,...,89,11.2,0,10,85,50,15,3,46064,22394
4,22,4.9,4.9,19,34,8.4,30,16,14,13,...,483,11.7,0,11,86,59,17,3,50412,57826


In [39]:
X_droprate_train, X_droprate_test, y_droprate_train, y_droprate_test = train_test_split(X_droprate, dummy_y, random_state=78)

In [40]:
X_droprate_scaler = scaler.fit(X_droprate_train)

# Scaling the data.
X_droprate_train_scaled = X_droprate_scaler.transform(X_droprate_train)
X_droprate_test_scaled = X_droprate_scaler.transform(X_droprate_test)

In [41]:
nn_droprate = tf.keras.models.Sequential()
input_features_droprate = len(X_droprate_train_scaled [0])
droprate_units_layer1 = input_features_droprate*2
nn_droprate.add(tf.keras.layers.Dense(units=droprate_units_layer1, activation="relu", input_dim=input_features_droprate))
nn_droprate.add(tf.keras.layers.Dense(units =4, activation = "relu"))
nn_droprate.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [42]:
drop_model = nn_droprate.fit(X_droprate_train_scaled, y_droprate_train, epochs = 50)
#evaluate model accuracy
model_loss_drop, model_accuracy_drop =nn_droprate.evaluate(X_droprate_test_scaled, y_droprate_test, verbose=2)
print(f"Loss: {model_loss_drop}, Accuracy: {model_accuracy_drop}")

Train on 2209 samples
Epoch 1/50
2209/2209 [==============================] - 1s 386us/sample - loss: 5.1877 - accuracy: 0.3671
Epoch 2/50
2209/2209 [==============================] - 0s 74us/sample - loss: 2.7406 - accuracy: 0.4260
Epoch 3/50
2209/2209 [==============================] - 0s 68us/sample - loss: 1.8731 - accuracy: 0.4541
Epoch 4/50
2209/2209 [==============================] - 0s 70us/sample - loss: 1.5347 - accuracy: 0.4703
Epoch 5/50
2209/2209 [==============================] - 0s 69us/sample - loss: 1.3449 - accuracy: 0.4812
Epoch 6/50
2209/2209 [==============================] - 0s 67us/sample - loss: 1.2761 - accuracy: 0.4971
Epoch 7/50
2209/2209 [==============================] - 0s 75us/sample - loss: 1.2288 - accuracy: 0.5016
Epoch 8/50
2209/2209 [==============================] - 0s 68us/sample - loss: 1.2041 - accuracy: 0.5093
Epoch 9/50
2209/2209 [==============================] - 0s 69us/sample - loss: 1.1796 - accuracy: 0.5170
Epoch 10/50
2209/2209 [=========